## This script sets up the model and performs all relevant model runs.

The time resolution is set using the nhours argument in the mkwargs dictionary. Use 1 for full detail.

In [1]:
import sys, os

import numpy as np
import pandas as pd

import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB
import grimsel.auxiliary.aux_sql_func as aql
import grimsel.config as config
import grimsel.model_loop_modifier as model_loop_modifier
import grimsel.analysis.sql_analysis as sql_analysis

db = config.DATABASE

sc_out = 'out_replace_all_test'

module 'grimsel.config_local' has no attribute 'PATH_CSV'
Using default csv path C:\Users\ashreeta\Documents\Martin\GRIMSEL_SOURCE\grimsel\input_data


In [2]:
connect_dict = dict(db=db,
                    password=config.PSQL_PASSWORD,
                    user=config.PSQL_USER,
                    port=config.PSQL_PORT,
                    host=config.PSQL_HOST)
sqlc = aql.sql_connector(**connect_dict)

# additional kwargs for the model
mkwargs = {
           'slct_encar': ['EL'],
           'slct_node': ['AT0', 'IT0', 'DE0', 'CH0', 'FR0'],
           'nhours': 1,
           'verbose_solver': True,
           'constraint_groups': MB.get_constraint_groups(excl=['chp', 'ror'])
           }

# additional kwargs for the i/o
iokwargs = {'resume_loop': False,
            'autocomplete_curtailment': True,
           }

nvr, nst = 30, 31
nsteps_default = [
                  ('swvr', nvr, np.arange),     # select vre share
                  ('swst', nst, np.arange),     # select storage capacity
                  ('swtc', 3, np.arange),       # select storage tech
                  ('swpt', 3, np.arange),       # select vre type
                  ('swyr', 5, np.arange),       # select meteo year
                  ('swco', 3, np.arange),       # select co2 emission price
                  ('swtr', 2, np.arange),       # cross-border transmission on/off
                  ('swrc', 26, np.arange)]      # select ramping cost

mlkwargs = {
            'sc_out': sc_out,
            'db': db,
            'nsteps': nsteps_default,
            'sql_connector': sqlc,
            }

sc_out = mlkwargs['sc_out']

ml = model_loop.ModelLoop(**mlkwargs, mkwargs=mkwargs, iokwargs=iokwargs)


swst_max = nst
dict_st = {nst: st for nst, st in enumerate(list(np.linspace(0, 0.3, swst_max)))}
swvr_max = nvr - 1
dict_vre = {nvr: vr for nvr , vr
            in enumerate(['default'] + list(np.linspace(0, 0.7, swvr_max)))}
dict_rc = {**{ii: ii/10 for ii in range(26)}, **{0: 1, 10: 0}}



self.slct_encar=['EL']
self.slct_pp_type=[]
self.slct_node=['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
self.nhours=1
self.constraint_groups=['capacity', 'charging_level', 'chp_new', 'energy_aggregation', 'energy_constraint', 'hydro', 'monthly_total', 'objective', 'ramp_rate', 'supply', 'transmission', 'variables', 'yearly_cost']
Output schema:  out_replace_all_test ; resume loop= False

~~~~~~~~~~~~~~~   WARNING:  ~~~~~~~~~~~~~~~~
You are about to delete existing schema out_replace_all_test.
The maximum run_id is 0.

Hit enter to proceed.

Dropping output schema:  out_replace_all_test
Reading input table def_month 
Reading input table def_week 
Reading input table parameter_month 
Reading input table def_plant 
Reading input table def_node filtered by nd in ['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
Reading input table def_pp_type 
Reading input table def_encar filtered by ca in ['EL']
Reading input table def_plant filtered by nd_id in [0, 1, 2, 3, 4], pt_id in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 

Assigning parameter chpprof... ok.
Assigning parameter supprof... ok.
Assigning parameter inflowprof... ok.
Assigning parameter priceprof... failed: set(s) ['fl_prof'] is/are empty or None.
Hydro parameters
Assigning parameter min_erg_mt_out_share... ok.
Assigning parameter max_erg_mt_in_share... ok.
Assigning parameter min_erg_share... ok.
Defining general parameters
Assigning parameter weight... ok.
Assigning parameter month_weight... ok.
Assigning parameter dmnd_sum... ok.
Assigning parameter grid_losses... ok.
Assigning parameter vc_dmnd_flex... ok.
Assigning parameter chp_cap_pwr_leg... ok.
Assigning parameter cap_trm_leg... ok.
Defining ramping parameters
Assigning parameter vc_ramp... ok.
Defining pp parameters
Assigning parameter ca_share_min... failed (column doesn't exist).
Assigning parameter ca_share_max... failed (column doesn't exist).
Assigning parameter pp_eff... ok.
Assigning parameter cf_max... ok.
Defining fuel parameters
Assigning parameter erg_inp... ok.
Assigning 

## Reduce output
Don't write hourly data.

In [3]:
ml.io.var_sy = []#[par for par in ml.io.var_sy if 'pwr' in par[0] or 'pwr_st_ch' in par[0]]
ml.io.par = [par for par in ml.io.par if not 'prof' in par[0]]
ml.io.var_tr = [var for var in ml.io.var_tr if 'erg' in var[0]]
ml.io.dual = []
print(ml.io.var_sy)

[]


## Some filtering of the ml.df_def_loop table

The ModelLoop object created a table with all combinations of the nsteps_default as model runs. 

Here this table is filtered so we don't end up doing 6'528'600 model runs:

In [4]:
# figure 14: emissions
slct_vr = list(np.arange(0, max([vrid for vrid, vrvl in dict_vre.items() if not vrvl == 'default' and vrvl <= 0.45]), 2) + 1)
slct_st = [0, 20]
mask_emissions = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 13: consecutive replacement
slct_vr = list(np.arange(0, max([vrid for vrid, vrvl in dict_vre.items() if not vrvl == 'default' and vrvl <= 0.425]), 1) + 1)
slct_st = [0, 20]#list(np.arange(0, ml.df_def_loop.swvr_id.max() + 5, 10))
mask_consec = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0, 1, 2]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0, 1]) &
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 12: qualitatively diverging storage impact
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.5, 0.7, 0.4]]
mask_years = (ml.df_def_loop.swvr_id.isin(slct_vr) &
#                ml.df_def_loop.swst_id.isin(slct_st) & --> all
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
#                ml.df_def_loop.swyr_id.isin([0]) & --> all
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )



# figure 11: ramping costs
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.3, 0.5, 0.7]]
slct_st = [0, 20]#list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_ramping = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) 
#               ml.df_def_loop.swrc_id.isin([0]) --> all
               )


# figures 9+10: various nuclear power indicators france 
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 4) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_base = (ml.df_def_loop.swvr_id.isin(slct_vr) &
               ml.df_def_loop.swst_id.isin(slct_st) &
               ml.df_def_loop.swtc_id.isin([0, 1]) &
               ml.df_def_loop.swpt_id.isin([0]) &
               ml.df_def_loop.swyr_id.isin([0]) &
               ml.df_def_loop.swco_id.isin([0]) &
               ml.df_def_loop.swtr_id.isin([0]) &
               ml.df_def_loop.swrc_id.isin([0]))


mask_total = mask_base | mask_ramping | mask_years | mask_consec | mask_emissions

ml.df_def_loop = pd.concat([ml.df_def_loop.loc[mask_total]])
ml.df_def_loop

,run_id,swvr_id,swst_id,swtc_id,swpt_id,swyr_id,swco_id,swtr_id,swrc_id,swvr,...,swtr,swrc,swvr_vl,swst_vl,swtc_vl,swpt_vl,swyr_vl,swco_vl,swtr_vl,swrc_vl
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Model loop

The ModelLoopModifier instance makes changes to ModelBase object as specified by the model run and its index combination.

In [5]:
from importlib import reload

reload(model_loop_modifier)

# init ModelLoopModifier
mlm = model_loop_modifier.ModelLoopModifier(ml)

# starting row of loop
irow_0 = ml.io.resume_loop if ml.io.resume_loop else 0

irow = irow = 0

ml.m._limit_prof_to_cap()
#ml.perform_model_run(zero_run=True)
# %
for irow in list(range(irow_0, len(ml.df_def_loop))):
    run_id = irow

    print('select_run')
    ml.select_run(run_id)

    ###
    print('set_trm_cap_onoff')
    mlm.set_trm_cap_onoff()
    
    ####
    print('set_co2_price')
    mlm.set_co2_price()

    ####
    print('set_winsol_year')
    mlm.set_winsol_year()

    ####
    print('select_vre_pp_types')
    slct_pp, slct_pt = mlm.select_vre_pp_types()

    ####
    print('scale_vre')
    mlm.scale_vre(slct_pt, dict_vre)

    ####
    print('set_ramping_cost')
    mlm.set_ramping_cost(dict_rc)

    ####
    print('select_storage_tech')
    mlm.select_storage_tech()

    ####
    print('set_storage_cap')
    mlm.set_storage_cap(dict_st)

    ############### RUN MODEL ###############
    print('fill_peaker_plants')
    ml.m.fill_peaker_plants(demand_factor=2)

    print('_limit_prof_to_cap')
    ml.m._limit_prof_to_cap()

    print('perform_model_run')
    ml.perform_model_run()

    for fn in [fn for fn in os.listdir('.') if 'ephemeral' in fn or 'def_loop' in fn]:
        os.remove(fn)





  36  5.2477184e+010  4.9173721e+010 8.45e-002 1.54e-001 9.24e-003
  37  5.2155528e+010  4.9464978e+010 5.88e-002 1.07e-001 6.38e-003
  38  5.1885059e+010  4.9724571e+010 2.32e-002 4.14e-002 2.48e-003
  39  5.1504110e+010  5.0041919e+010 1.40e-002 1.97e-002 1.18e-003
  40  5.1343283e+010  5.0176474e+010 1.00e-002 1.31e-002 7.81e-004
  41  5.1227634e+010  5.0274286e+010 6.55e-003 7.69e-003 4.60e-004
  42  5.1208276e+010  5.0289057e+010 3.94e-003 4.41e-003 2.63e-004
  43  5.1207680e+010  5.0289652e+010 3.88e-003 4.34e-003 2.59e-004
  44  5.1179543e+010  5.0313094e+010 3.40e-003 3.62e-003 2.16e-004
  45  5.0941298e+010  5.0506827e+010 7.42e-003 5.14e-004 3.06e-005
  46  5.0928201e+010  5.0517552e+010 5.93e-003 4.10e-004 2.43e-005
  47  5.0866942e+010  5.0566454e+010 6.12e-003 2.58e-004 1.52e-005
  48  5.0841830e+010  5.0586650e+010 4.16e-003 1.54e-004 8.92e-006
  49  5.0809493e+010  5.0612411e+010 5.02e-003 9.70e-005 5.52e-006
  50  5.0776984e+010  5.0638427e+010 4.50e-003 2.23e-005 1.05e


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.7.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\ashreeta\AppData\Local\Temp\manual_log_file_replace_all_test.cplex.sol' open.
CPLEX> Problem 'ephemeralacgb.pyomo.lp' read.
Read time = 7.30 sec. (409.44 ticks)
CPLEX> Solution file 'C:\Users\ashreeta\AppData\Local\Temp\manual_soln_file_replace_all_test_0.cplex.sol' read.
CPLEX> Problem name         : ephemeralacgb.pyomo.lp
Objective sense      : Minimize
Variables            : 1893124  [Nneg: 1507665,  Free: 385459,  Qobj: 131400]
Objective nonzeros   :  131605
Objective Q nonzeros :  131400
Linear constraints   : 2602742  [Less: 1576806,  Greater: 131460,  Equal: 894476]
  Nonzeros 

Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.007 sec
Writing to db:  out_replace_all_test.var_mt_erg_mt... done in 0.076 sec
Component share_ws_set does not exist... skipping.
Component co2_int does not exist... skipping.
Writing to db:  out_replace_all_test.par_cap_pwr_leg... done in 0.011 sec
Writing to db:  out_replace_all_test.par_cap_trm_leg... done in 0.027 sec
Writing to db:  out_replace_all_test.par_cf_max... done in 0.012 sec
Writing to db:  out_replace_all_test.par_grid_losses... done in 0.004 sec
Component erg_max does not exist... skipping.
Component hyd_pwr_in_mt_max does not exist... skipping.
Component hyd_pwr_out_mt_min does not exist... skipping.
Writing to db:  out_replace_all_test.par_vc_dmnd_flex... done in 0.004 sec
Writing to db:  out_replace_all_test.par_factor_vc_fl_lin_0... done in 0.004 sec
Writing to db:  out_replace_all_test.par_factor_vc_fl_lin_1... done in 0.004 sec
Writing to db:  out_replace_all_test.par_factor_vc_co2_lin_0... done in 

  25  8.3586571e+010  1.3785215e+010 2.00e+002 3.70e+002 2.23e+001
  26  7.3712542e+010  2.5075172e+010 1.27e+002 2.34e+002 1.41e+001
  27  6.9618453e+010  2.9897835e+010 4.61e+001 8.50e+001 5.12e+000
  28  6.1040791e+010  3.9074024e+010 1.14e+001 2.10e+001 1.27e+000
  29  5.9697174e+010  4.0555643e+010 8.14e+000 1.50e+001 9.04e-001
  30  5.7467561e+010  4.2897590e+010 4.04e+000 7.44e+000 4.48e-001
  31  5.5835023e+010  4.4494457e+010 1.60e+000 2.95e+000 1.78e-001
  32  5.4273935e+010  4.5921117e+010 1.05e+000 1.94e+000 1.17e-001
  33  5.3744342e+010  4.6427135e+010 7.93e-001 1.46e+000 8.81e-002
  34  5.3106676e+010  4.7022613e+010 5.56e-001 1.02e+000 6.17e-002
  35  5.2515715e+010  4.7569377e+010 3.74e-001 6.90e-001 4.15e-002
  36  5.2105636e+010  4.7950596e+010 2.65e-001 4.89e-001 2.94e-002
  37  5.1669951e+010  4.8355428e+010 1.65e-001 3.02e-001 1.82e-002
  38  5.1642615e+010  4.8376506e+010 1.27e-001 2.32e-001 1.40e-002
  39  5.1150864e+010  4.8798941e+010 7.77e-002 1.41e-001 8.48e


          create_tempfile is monkey patched
          
Solver script file=ephemeralfhff.cplex.script
Solver warm-start file=C:\Users\ashreeta\AppData\Local\Temp\manual_soln_file_replace_all_test_0.cplex.sol
Solver log file: 'C:\Users\ashreeta\AppData\Local\Temp\manual_log_file_replace_all_test.cplex.sol'
Solver solution file: 'C:\Users\ashreeta\AppData\Local\Temp\manual_soln_file_replace_all_test_1.cplex.sol'
Solver problem files: ('ephemeralgigh.pyomo.lp',)

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.7.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2017.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\ashreeta\AppData\Local\Temp\manual_log_file_replace_all_test.cplex.sol' open.
CPLEX> Problem 'ephemeralgigh.pyomo.lp' read.
Rea

CPLEX> Solution written to file 'C:\Users\ashreeta\AppData\Local\Temp\manual_soln_file_replace_all_test_1.cplex.sol'.
CPLEX> Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.014 sec
Writing to db:  out_replace_all_test.var_yr_erg_fl_yr... done in 0.012 sec
Writing to db:  out_replace_all_test.var_yr_pwr_ramp_yr... done in 0.009 sec
Writing to db:  out_replace_all_test.var_yr_vc_fl_pp_yr... done in 0.009 sec
Writing to db:  out_replace_all_test.var_yr_vc_ramp_yr... done in 0.008 sec
Writing to db:  out_replace_all_test.var_yr_vc_co2_pp_yr... done in 0.006 sec
Writing to db:  out_replace_all_test.var_yr_vc_om_pp_yr... done in 0.011 sec
Writing to db:  out_replace_all_test.var_yr_fc_om_pp_yr... done in 0.011 sec
Component fc_cp_pp_yr does not exist... skipping.
Writing to db:  out_replace_all_test.var_yr_fc_dc_pp_yr... done in 0.010 sec
Writing to db:  out_replace_all_test.var_yr_vc_dmnd_flex_yr... done in 0.003 sec
Writing to db:  out_replace_all_test.var_yr_cap_pwr_rem...

   7  1.2254331e+015 -3.2161330e+015 1.30e+011 2.38e+011 1.45e+010
   8  8.4435343e+013 -9.1917591e+014 2.72e+010 5.00e+010 3.05e+009
   9  1.2389305e+013 -3.2583529e+014 4.56e+009 8.38e+009 5.11e+008
  10  9.7959131e+012 -1.4055409e+014 9.98e+008 1.84e+009 1.12e+008
  11  1.1618868e+013 -1.3776617e+014 7.84e+008 1.44e+009 8.78e+007
  12  1.3676950e+013 -1.2000058e+014 4.92e+008 9.04e+008 5.51e+007
  13  1.7954378e+013 -1.0230007e+014 2.04e+008 3.75e+008 2.28e+007
  14  1.3786222e+013 -4.9860311e+013 8.07e+007 1.48e+008 9.04e+006
  15  1.4144205e+013 -4.6504612e+013 6.30e+007 1.16e+008 7.06e+006
  16  1.2468241e+013 -3.5494518e+013 3.78e+007 6.95e+007 4.24e+006
  17  1.1485848e+013 -2.4133810e+013 6.60e+006 1.21e+007 7.40e+005
  18  9.3915768e+012 -1.9091075e+013 4.72e+006 8.68e+006 5.29e+005
  19  7.1351634e+011 -9.9477847e+011 2.39e+005 4.39e+005 2.67e+004
  20  2.3269066e+011 -1.8959386e+011 2.98e+004 5.49e+004 3.34e+003
  21  1.1765913e+011 -2.5191616e+010 6.84e+003 1.26e+004 7.66e

Writing to db:  out_replace_all_test.par_dmnd... done in 5.278 sec
Writing to db:  out_replace_all_test.var_sy_pwr... done in 5.555 sec
Writing to db:  out_replace_all_test.par_erg_inp... done in 0.018 sec
Writing to db:  out_replace_all_test.par_erg_chp... done in 0.017 sec
Writing to db:  out_replace_all_test.par_capchnge_max... done in 0.003 sec
Writing to db:  out_replace_all_test.par_price_co2... done in 0.017 sec
Writing to db:  out_replace_all_test.par_vc_fl... done in 0.213 sec
Writing to db:  out_replace_all_test.par_cap_avlb... done in 0.140 sec
Writing to db:  out_replace_all_test.var_tr_erg_trm_rv_yr... done in 0.008 sec
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.005 sec
Writing to db:  out_replace_all_test.var_tr_erg_trm_sd_yr... done in 0.007 sec
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.004 sec
select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storag

  54  4.9334283e+010  4.9281748e+010 4.24e-003 2.29e-005 1.08e-006
  55  4.9327975e+010  4.9286897e+010 4.15e-003 1.68e-005 6.89e-007
  56  4.9324942e+010  4.9289388e+010 3.93e-003 1.53e-005 5.44e-007
  57  4.9320339e+010  4.9293173e+010 3.83e-003 9.22e-006 2.37e-007
  58  4.9318000e+010  4.9295088e+010 2.63e-003 6.73e-006 9.85e-008
  59  4.9314409e+010  4.9298089e+010 5.53e-003 6.53e-006 8.05e-008
  60  4.9313027e+010  4.9299247e+010 5.62e-003 6.88e-006 6.81e-008
  61  4.9311872e+010  4.9300209e+010 4.64e-003 6.57e-006 5.43e-008
  62  4.9310399e+010  4.9301460e+010 8.33e-003 6.74e-006 5.06e-008
  63  4.9309977e+010  4.9301808e+010 6.29e-003 6.86e-006 4.68e-008
  64  4.9308735e+010  4.9302873e+010 9.48e-003 6.48e-006 5.28e-008
  65  4.9308000e+010  4.9303502e+010 9.00e-003 6.28e-006 4.55e-008
  66  4.9306651e+010  4.9304663e+010 1.72e-002 5.36e-006 8.23e-008
  67  4.9306306e+010  4.9304971e+010 2.72e-002 6.01e-006 5.67e-008
  68  4.9305991e+010  4.9305252e+010 3.97e-002 5.83e-006 3.35e

KeyboardInterrupt: 

## Basic analysis tables
Prints the executed SQL query.

In [ ]:
sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db)
print(sqa.build_tables_plant_run_quick())
print(sqa.build_table_plant_run_tot_balance(from_quick=True))

## Advanced analysis

Analysis related to emissions and hourly filtered output requires writing all hourly data.

In [ ]:
# EMISSIONS LIN

slct_run_id = aql.exec_sql('''
                           SELECT run_id FROM {sc}.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 5) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
                           AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           '''.format(sc=sc_out), db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db, slct_run_id=slct_run_id)
sqa.analysis_emissions_lin()

In [ ]:
# FILTDIFF FOR FRENCH NUCLEAR
import grimsel.auxiliary.maps as maps

mps = maps.Maps(sc_out, db)

slct_nd_id = [mps.dict_nd_id[nd] for nd in ['FR0']]
slct_run_id = aql.exec_sql('''
                           SELECT run_id FROM {sc}.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 10) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
                           AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           AND swco_vl = '40EUR/tCO2'
                           '''.format(sc=sc_out), db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

reload(sql_analysis)
sqac_filt_chgdch = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db,
                                            slct_pt=['NUC_ELC'],
                                            slct_run_id=slct_run_id,
                                            nd_id=slct_nd_id)
self = sqac_filt_chgdch
sqac_filt_chgdch.analysis_agg_filtdiff()
